In [1]:
import numpy as np
import torch

from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_sample_weight
from numpy import random

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

import os
from pathlib import Path
home = str(Path.home())

In [2]:
def LR(X_train, y_train, X_test, y_test):
    scaler  = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test  = scaler.transform(X_test)
    clf = LogisticRegressionCV(n_jobs=-1)
    clf.fit(X_train, y_train)
    y_hat =    clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    
    return y_hat, y_hat_pr

In [3]:
def RF(X_train, y_train, X_test, y_test):
    clf = RandomForestClassifier(n_estimators=300, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_hat =    clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    
    return y_hat, y_hat_pr

In [9]:
factor = 0.8
nr_mean = 3
mean_tables = []
nr_samples=2000

datasets = ['cifar10', 'cifar100', 'imagenet']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
detectors = ['lid', 'multilid']

results = {}

for it in range(nr_mean):
    random_state = [21, 30, 65][it] # random.randint(100)
    print("random state: ", random_state)
    final_table = np.zeros((len(attacks), 2))
    
    results[it] = {}
    
    for dataset in datasets:
        
        if dataset == 'imagenet':
            k=30
        else:
            k=20
        
        for model in models:
            
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
                
            if not dataset in results[it]:
                results[it][dataset] = {}
            if not model in results[it][dataset]:
                results[it][dataset][model] = {}
                
            #print("config: ", random_state, dataset, model)
            
            for detector in detectors:
                base_path = os.path.join(home, 'workspace/multiLID/data/extract/run_{}/'.format(it+1))
                nor_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_normalos_8255.pt"))
                adv_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_adverlos_8255.pt"))
                nor_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_normalos_8255.pt"))
                adv_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_adverlos_8255.pt"))
                nor_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_normalos_8255.pt"))
                adv_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_adverlos_8255.pt"))
                nor_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_normalos_8255.pt"))
                adv_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_adverlos_8255.pt"))
                nor_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_normalos.pt"))
                adv_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_adverlos.pt"))
                nor_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_normalos.pt"))
                adv_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_adverlos.pt"))

                nor = [nor_fgsm, nor_bim, nor_pgd, nor_aa, nor_df, nor_cw]
                adv = [adv_fgsm, adv_bim, adv_pgd, adv_aa, adv_df, adv_cw]

                for rows in range(len(nor)):
                    X_nor = nor[rows]
                    X_adv = adv[rows]
                    if len(X_nor.shape) > 2: 
                        X_nor = nor[rows].reshape((nor[rows].shape[0], -1))
                        X_adv = adv[rows].reshape((adv[rows].shape[0], -1))

                    y_nor = np.zeros(X_nor.shape[0]).astype('int')
                    y_adv = np.ones(X_nor.shape[0]).astype('int')

                    x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(X_nor, y_nor, test_size=1-factor, train_size=factor, random_state=random_state)
                    x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(X_adv, y_adv, test_size=1-factor, train_size=factor, random_state=random_state)

                    X_train = np.concatenate((x_train_n, x_train_a))
                    y_train = np.concatenate((y_train_n, y_train_a))

                    X_test = np.concatenate((x_test_n, x_test_a))
                    y_test = np.concatenate((y_test_n, y_test_a))

                    if detector == 'lid':
                        y_hat, y_hat_pr = LR(X_train, y_train, X_test, y_test)
                    elif detector == 'multilid':
                        y_hat, y_hat_pr = RF(X_train, y_train, X_test, y_test)

                    auc = round(100*roc_auc_score(y_test, y_hat_pr), 2)
                    f1 =  round(100*f1_score(y_test, y_hat), 2)
     
                    if not detector in results[it][dataset][model]:
                        results[it][dataset][model][detector] = {}
                    
                    if not attacks[rows] in results[it][dataset][model][detector]: 
                        results[it][dataset][model][detector][attacks[rows]] = {}
                    
                    
                    if not 'auc' in results[it][dataset][model][detector][attacks[rows]]:
                        results[it][dataset][model][detector][attacks[rows]]['auc'] = {}
                    if not 'f1' in results[it][dataset][model][detector][attacks[rows]]:
                        results[it][dataset][model][detector][attacks[rows]]['f1' ] = {}
                        
                    results[it][dataset][model][detector][attacks[rows]]['auc'] = auc
                    results[it][dataset][model][detector][attacks[rows]]['f1'] = f1                    

                    # print(attacks[rows].upper(), '&' , auc, '&', f1)

random state:  21
random state:  30
random state:  65


In [10]:
lid_list = []

for dataset in ['cifar10', 'cifar100', 'imagenet']:
    for model in ['wrn28-10', 'vgg16', 'wrn50-2']:
        if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
            continue

        if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
            continue
        
        tmp = np.zeros((6, 2))
        for it, val in enumerate(results[0][dataset][model]['lid'].items()):
            #print(it, val)
            tmp[it, 0] = val[1]['auc'].copy()
            tmp[it, 1] = val[1]['f1'].copy()
        lid_list.append(tmp.copy())

result_table = np.concatenate(lid_list, axis=1)
for row in result_table:
    for i, val in enumerate(row):
        print(val, end='')
        if (i+1) % 1 == 0 and i != len(row)-1:
            print(' & ', end='')
        elif i != len(row)-1:
            print(' ', end='')
    print(" \\\ ")

99.0 & 93.02 & 87.62 & 79.74 & 99.05 & 94.89 & 77.36 & 71.16 & 72.83 & 66.16 \\ 
87.04 & 80.59 & 91.14 & 83.69 & 96.04 & 88.66 & 83.62 & 76.75 & 95.09 & 88.53 \\ 
88.34 & 78.73 & 91.2 & 83.27 & 98.1 & 92.6 & 83.7 & 76.96 & 95.31 & 88.0 \\ 
97.32 & 91.56 & 97.5 & 92.25 & 99.51 & 96.59 & 87.02 & 78.87 & 99.95 & 99.75 \\ 
95.53 & 87.95 & 86.99 & 75.82 & 56.5 & 52.73 & 54.58 & 54.01 & 54.86 & 51.38 \\ 
94.5 & 85.35 & 83.87 & 75.64 & 54.66 & 51.79 & 61.89 & 60.24 & 54.19 & 47.58 \\ 


In [ ]:
lid_list = []

for dataset in ['cifar10', 'cifar100', 'imagenet']:
    for model in ['wrn28-10', 'vgg16', 'wrn50-2']:
        if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
            continue

        if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
            continue
        
        tmp = np.zeros((6, 2))
        for it, val in enumerate(results[0][dataset][model]['multilid'].items()):
            
            print(it, val)
            tmp[it, 0] = val[1]['auc'].copy()
            tmp[it, 1] = val[1]['f1'].copy()
        lid_list.append(tmp.copy())

result_table = np.concatenate(lid_list, axis=1)
for row in result_table:
    for i, val in enumerate(row):
        print(val, end='')
        if (i+1) % 1 == 0 and i != len(row)-1:
            print(' & ', end='')
        elif i != len(row)-1:
            print(' ', end='')
    print(" \\\ ")